# Imports

In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import pandas as pd
from connection_keys import connection, alchemy_connection

### Setup para abrir o navegador com o Selenium e puxar a primeira página

In [4]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--disable-extensions')
#options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)

In [5]:

#--| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

### Configuiração para percorer as páginas e salvar todos dados com a taga referente ao produto em uma lista

In [6]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

### Tranformação da lista de listas em uma lista normal para percorer com mais facildiade os atributos do produto

In [7]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

### Percorrendo a lista, capurando e salvando cada atributo no dicionario

In [8]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)


### Transformando o dict em Dataframe e visualizando uma amostra dos dados

In [9]:
df = pd.DataFrame(produtos)
df.sample(10)

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
295,Evga,Placa de Vídeo EVGA NVIDIA GeForce GTX 1060 3G...,"1.164,59","999,88","849,90",Indisponivel,2020-09-16,20:06:05,Kabum
118,Gainward,Placa de Vídeo Gainward NVIDIA GeForce RTX 208...,0,"7.016,35","5.963,90",Disponivel,2020-09-16,20:06:05,Kabum
133,Rise Mode,Suporte para Placa de Vídeo Rise Mode Vermelho...,"30,47","29,29","24,90",Disponivel,2020-09-16,20:06:05,Kabum
178,Evga,Placa de Video EVGA GeForce RTX 2060 Super SC ...,"4.117,53","3.529,29","2.999,90",Disponivel,2020-09-16,20:06:05,Kabum
356,Amd,"Placa de Vídeo AMD Radeon Pro WX 9100, 16GB, H...",0,"12.470,47","10.599,90",Indisponivel,2020-09-16,20:06:05,Kabum
55,Evga,Placa de Video EVGA GeForce RTX 2070 Super XC ...,0,"5.294,00","4.499,90",Disponivel,2020-09-16,20:06:05,Kabum
252,Galax,Placa de Vídeo Galax NVIDIA GeForce RTX 2070 S...,"4.588,12","4.164,59","3.539,90",Indisponivel,2020-09-16,20:06:05,Kabum
381,Powercolor,"Placa de Vídeo PowerColor AMD Radeon RX 550, 4...",0,"770,47","654,90",Indisponivel,2020-09-16,20:06:05,Kabum
54,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2070 Su...,"4.917,53","4.352,82","3.699,90",Disponivel,2020-09-16,20:06:05,Kabum
17,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce RTX 206...,"2.941,06","2.763,41","2.348,90",Disponivel,2020-09-16,20:06:05,Kabum



### Criando a conexão com o banco e salvando os dados

In [11]:
engine = alchemy_connection('db_hardware')
df.to_sql('graphic_card', con=engine, if_exists='append', index = False)

----------------------------------------------------------------------------------------------------

In [13]:
browser2 = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
#--| Parse or automation
browser2.get('https://www.pichau.com.br/hardware/placa-de-video?p=1')
web = soup(browser2.page_source, 'html.parser')

In [18]:
pagina = 0
html_produtos = []
flag = 0

while pagina != 5:
    items = web.findAll('div', {"class":"product-item-info"})
    if items != [] | item:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.pichau.com.br/hardware/placa-de-video?p={pagina}')
            web = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

In [15]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

In [35]:
items[15].div

<div class="product details product-item-details">
<strong class="product name product-item-name">
<a class="product-item-link" href="https://www.pichau.com.br/hardware/placa-de-video-gigabyte-radeon-rx-5700-8gb-gaming-oc-256-bit-gv-r57gaming-oc-8gd">
Placa de Video Gigabyte Radeon RX 5700 8GB Gaming OC 256-bit, GV-R57GAMING OC-... </a>
</strong>
<div class="price-box price-final_price" data-price-box="product-id-17732" data-product-id="17732" data-role="priceBox">
<span class="price-container price-final_price tax">
<span class="price-wrapper" data-price-amount="3238.59" data-price-type="finalPrice" id="product-price-17732">
<span class="price">R$3.238,59</span> </span>
<span class="price-installments">
<span>10x de R$323,86</span> <strong>sem juros</strong> no cartão </span>
</span>
<span class="price-boleto">
<span>à vista R$2.849,96</span> no boleto com <strong>12% de desconto</strong></span>
</div>
<div class="product-item-inner">
<div class="product actions product-item-actions">

In [ ]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)

In [15]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
# https://www.kabum.com.br/cgi-local/site/listagem/listagem.cgi?string=processador&btnG=&pagina=1&ordem=5&limite=30&prime=false&marcas=[%225%22,%226%22]&tipo_produto=[]&filtro=[]

In [16]:
# images = html.findAll('div', {"class":"sc-fznKkj fEpBwY"})

# # images.img['src']
# for image in images:
#     print(image.img['src'])
#     # if "indisponivel" in image.img['src']:
#     #     print('Indisponivel')
#     # elif "disponivel" in image.img['src']:
#     #     print('Disponivel')

In [17]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [18]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)